# Setup

In [ ]:
import create_models
from topodiff.resample import UniformSampler
from topodiff.image_datasets_diffusion_model import load_data
import torch

# Model

In [ ]:
image_size = 64

mean_variance = create_models.mean_variance(
    image_size=image_size,
    num_channels=128,
    num_res_blocks=3,
    channel_mult="",
    learn_sigma=True,
    use_checkpoint=False,
    attention_resolutions="16,8",
    num_heads=4,
    num_head_channels=-1,
    num_heads_upsample=-1,
    use_scale_shift_norm=True,
    dropout=0.3,
    resblock_updown=False,
    use_fp16=True,
    use_new_attention_order=False,
)
diffusion = create_models.gaussian_diffusion(
    steps=1000,
    learn_sigma=True,
    sigma_small=False,
    noise_schedule="cosine",
    use_kl=False,
    predict_xstart=False,
    rescale_timesteps=False,
    rescale_learned_sigmas=False,
    timestep_respacing="",
)
schedule_sampler = UniformSampler(diffusion)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
mean_variance.to(torch.cuda.current_device())
torch.Tensor(1).to(device)

# Data

In [ ]:
data = load_data(
  data_dir=r".\data\dataset_1_diff\training_data",
  batch_size=32,
  image_size=image_size,
)

# Training

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

image_batch, constraint_batch, _ = next(data)
